In [85]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import glob
import os

In [86]:
df_list = []
os_list = os.listdir()
os_list.sort()

csv_list = [file for file in os_list if file.endswith('.csv')]
csv_list.pop(0)
json_list = [file for file in os_list if file.endswith('.json')]

for i,val in enumerate(csv_list):
    
    # open json and create dict from file
    with open(json_list[i]) as json_data:
        data = json.load(json_data)
    
    df_json = pd.DataFrame(data['selectedAttributes'])
    
    lga_name_to_title = dict(zip(df_json['name'],df_json['title']))
    
    # open csv file and clean
    df = pd.read_csv(csv_list[i], index_col=None, header=0)
    
    clean_cols = []

    for col in df.columns:
        clean_cols.append(lga_name_to_title[col.strip(' ')].split(' Age')[0].replace('Persons ','').replace('Females ','').replace('Males ','').replace('emales ','').replace('ales ',''))
    
    df.columns = clean_cols
    df = df.astype({"LGA Code 2016": str}) 
    #
    
    
    
    #
    df = df.groupby(df.columns, axis=1).sum()
    
    col_list = df.columns

    for col in col_list:
        if (('Total' in col) and ('LGA' not in col)):
            df=df.drop(col,axis=1)
            
    df_list.append(df)

In [87]:
df = pd.concat(df_list,axis=1)

In [88]:

df = df.groupby(df.columns, axis=1).sum()

In [89]:
df = df.rename(columns = {'China excludes SARs and Taiwan':'China', 
                         'Korea Republic of South':'South Korea',
                         'South Eastern Europe nfd':'South Eastern Europe',
                         'The Former Yugoslav Republic of Macedonia':'North Macedonia',
                         'Hong Kong SAR of China':'Hong Kong',
                         'LGA Name 2016':'LGA Name'})

In [90]:
df = df.drop(columns = ['Born elsewhere','Country of birth not stated','LGA Code 2016'])

In [91]:
second_col = df.pop('LGA Name')


In [92]:
df

,Afghanistan,Australia,Bangladesh,Bosnia and Herzegovina,Cambodia,Canada,Chile,China,Croatia,Egypt,...,South Eastern Europe,Sri Lanka,Taiwan,Thailand,North Macedonia,Turkey,United States of America,Vietnam,Wales,Zimbabwe
0,34,83170,170,0,10,80,17,238,89,28,...,62,200,3,125,0,0,191,65,78,58
1,0,46805,108,0,0,99,15,558,0,0,...,0,120,80,43,0,0,310,93,67,45
2,0,67975,3,0,12,163,58,108,19,12,...,8,41,9,124,0,9,282,43,101,58
3,0,3744,0,0,0,0,0,0,0,0,...,0,0,8,12,0,0,0,0,0,0
4,36,69062,65,3,12,108,21,240,43,36,...,28,106,0,93,5,0,120,114,43,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,148,89904,567,47,124,363,47,1902,31,72,...,16,272,144,167,3,13,163,481,77,221
541,0,17429,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,8,0,0,0
542,0,13136,0,0,3,11,0,54,3,0,...,0,31,7,17,0,0,51,40,11,35
543,14,33715,0,0,124,46,8,59,9,0,...,12,14,14,113,4,0,70,341,31,115


In [93]:
for i,row in enumerate(df.values):
    df.iloc[i] = df.iloc[i].apply(lambda x: 100*(x/df.iloc[i].sum()))

In [94]:
df.head(50)

,Afghanistan,Australia,Bangladesh,Bosnia and Herzegovina,Cambodia,Canada,Chile,China,Croatia,Egypt,...,South Eastern Europe,Sri Lanka,Taiwan,Thailand,North Macedonia,Turkey,United States of America,Vietnam,Wales,Zimbabwe
0,0.036848,90.135686,0.184238,0.000000,0.010838,0.086700,0.018424,0.257933,0.096454,0.030345,...,0.067193,0.216750,0.003251,0.135469,0.000000,0.000000,0.206997,0.070444,0.084533,0.062858
1,0.000000,88.787086,0.204871,0.000000,0.000000,0.187799,0.028454,1.058502,0.000000,0.000000,...,0.000000,0.227635,0.151757,0.081569,0.000000,0.000000,0.588057,0.176417,0.127096,0.085363
2,0.000000,89.200184,0.003937,0.000000,0.015747,0.213897,0.076110,0.141723,0.024933,0.015747,...,0.010498,0.053802,0.011810,0.162719,0.000000,0.011810,0.370054,0.056427,0.132537,0.076110
3,0.000000,96.197328,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.205550,0.308325,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.047766,91.634270,0.086245,0.003981,0.015922,0.143299,0.027864,0.318442,0.057054,0.047766,...,0.037152,0.140645,0.000000,0.123396,0.006634,0.000000,0.159221,0.151260,0.057054,0.066342
5,0.000000,88.574109,0.005026,0.000000,0.010052,0.172561,0.030156,0.110573,0.036858,0.036858,...,0.055287,0.010052,0.005026,0.207743,0.000000,0.000000,0.407110,0.050261,0.140729,0.020104
6,0.000000,88.751200,0.000000,0.000000,0.000000,0.187625,0.039270,0.026180,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.056724,0.000000,0.026180,0.479972,0.026180,0.239986,0.069814
7,0.000000,92.877917,0.000000,0.000000,0.000000,0.135272,0.000000,0.094691,0.000000,0.094691,...,0.000000,0.216436,0.020291,0.040582,0.000000,0.000000,0.060873,0.000000,0.060873,0.020291
8,1.349792,59.801380,0.794806,0.167924,0.044648,0.098489,0.221108,1.529699,0.337653,0.642148,...,0.173833,1.609639,0.092744,0.204365,0.062048,0.436470,0.267398,0.322387,0.043663,0.104563
9,0.000000,95.689150,0.000000,0.000000,0.000000,0.117302,0.000000,0.097752,0.000000,0.000000,...,0.000000,0.058651,0.058651,0.000000,0.000000,0.000000,0.029326,0.058651,0.058651,0.078201


In [96]:
df1 = pd.read_csv('lga_g09a_country_of_birth_by_age_by_sex_census_2016-8577272101096579498.csv')

In [97]:
df.insert(0, 'LGA Name', second_col)
df.insert(1, 'LGA Code', df1[' lga_code_2016'])



In [98]:
for i,name in enumerate(df['LGA Name']):
    df['LGA Name'][i] = df['LGA Name'][i].split(')')[0]+')'

<ipython-input-98-bf6dee3ffc8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LGA Name'][i] = df['LGA Name'][i].split(')')[0]+')'


In [99]:
df['LGA Name'] = df['LGA Name'].apply(lambda x: x.split(' (')[0])

In [100]:
clean_cols = []

for col in df.columns:
    clean_cols.append(col.replace(' ','_').lower())

df.columns = clean_cols

In [102]:
df.to_csv('clean_country_of_birth_by_lga.csv',index=False)